In [1]:
import requests

In [2]:
class getWeather:

    data = None
    count = None

    def __init__(self, apiKey, city, unit):
        self.apiKey = apiKey
        self.unit = unit
        self.city = city

    def requestForecast(self, apiKey, city, unit, cast = 'forecast'):
        url = f'http://api.openweathermap.org/data/2.5/{cast}?appid={apiKey}&q={city}&mode=json&units={unit}' 

        response = requests.get(url).json()
        self.count = response['cnt']
        self.data = response

        print('New data has been retrieved')

        return response

    def printForecast(self):
        for day in range(self.count):
            date = self.data['list'][day]['dt_txt']
            temp = self.data['list'][day]['main']['temp']
            humidity = self.data['list'][day]['main']['humidity']
            windSpeed = self.data['list'][day]['wind']['speed']
            windDeg = self.data['list'][day]['wind']['deg']
            weather = self.data['list'][day]['weather'][0]['description']
            weatherId = self.data['list'][day]['weather'][0]['id']
            cloud = self.data['list'][day]['clouds']['all']

            print('\nDate/time:	{}'.format(date))
            print('Temperature: 	{} Celsius'.format(temp))
            print('Humidity:	{}%'.format(humidity))
            print('windspeed:	{} m/s'.format(windSpeed))
            print('windDeg:	{}'.format(windDeg))
            print('Clouds: 	{}%'.format(cloud))
            print('Weather ID: 	{}'.format(weatherId))
            print('Weather: 	{}'.format(weather))

In [4]:
apiKey = '##########################################'	#change key
unit = 'metric'
#city = input('Enter city: ')
city = 'Arnhem'
cast = 'forecast'

In [5]:
weatherData = getWeather(apiKey, city, unit) #not needed

In [6]:
data = weatherData.requestForecast(apiKey, city, unit) 


New data has been retrieved


In [7]:
weatherData.printForecast()


Date/time:	2021-06-10 12:00:00
Temperature: 	25.34 Celsius
Humidity:	42%
windspeed:	3.23 m/s
windDeg:	257
Clouds: 	15%
Weather ID: 	801
Weather: 	few clouds

Date/time:	2021-06-10 15:00:00
Temperature: 	24.94 Celsius
Humidity:	46%
windspeed:	2.67 m/s
windDeg:	252
Clouds: 	15%
Weather ID: 	801
Weather: 	few clouds

Date/time:	2021-06-10 18:00:00
Temperature: 	23.17 Celsius
Humidity:	60%
windspeed:	1.62 m/s
windDeg:	286
Clouds: 	28%
Weather ID: 	802
Weather: 	scattered clouds

Date/time:	2021-06-10 21:00:00
Temperature: 	16.79 Celsius
Humidity:	90%
windspeed:	2.47 m/s
windDeg:	346
Clouds: 	100%
Weather ID: 	804
Weather: 	overcast clouds

Date/time:	2021-06-11 00:00:00
Temperature: 	15.03 Celsius
Humidity:	92%
windspeed:	0.89 m/s
windDeg:	299
Clouds: 	78%
Weather ID: 	803
Weather: 	broken clouds

Date/time:	2021-06-11 03:00:00
Temperature: 	14.05 Celsius
Humidity:	93%
windspeed:	2.04 m/s
windDeg:	237
Clouds: 	20%
Weather ID: 	801
Weather: 	few clouds

Date/time:	2021-06-11 06:00:00
Tempe

In [8]:
from fastai.collab import *
from fastai.tabular.all import *

In [9]:
forecast_raw = pd.DataFrame(columns = ['date_time','temp','humid','wspd','wdeg','cloud','weather_id'])

In [10]:
number_of_rows = data['cnt']

for i in range(number_of_rows):
    date_time = data['list'][i]['dt_txt']
    date = date_time[:10]
    temp = data['list'][i]['main']['temp']
    humidity = data['list'][i]['main']['humidity']
    windSpeed = data['list'][i]['wind']['speed']
    windDeg = data['list'][i]['wind']['deg']
    weatherId = data['list'][i]['weather'][0]['id']
    cloud = data['list'][i]['clouds']['all']
    
    forecast_raw = forecast_raw.append({
        'date_time':date_time,
        'temp':temp,
        'humid':humidity,
        'wspd':windSpeed,
        'wdeg':windDeg,
        'cloud':cloud,
        'weather_id':weatherId
    }, ignore_index=True)

In [11]:
forecast_raw.head()

,date_time,temp,humid,wspd,wdeg,cloud,weather_id
0,2021-06-10 12:00:00,25.34,42,3.23,257,15,801
1,2021-06-10 15:00:00,24.94,46,2.67,252,15,801
2,2021-06-10 18:00:00,23.17,60,1.62,286,28,802
3,2021-06-10 21:00:00,16.79,90,2.47,346,100,804
4,2021-06-11 00:00:00,15.03,92,0.89,299,78,803


In [12]:
#remove data at night
for i in range(len(forecast_raw)):
    if int(forecast_raw['date_time'][i][11:13]) < 4 or int(forecast_raw['date_time'][i][11:13]) > 22: 
        forecast_raw.drop(index=i, axis=0, inplace=True)

In [13]:
forecast_raw.reset_index(inplace=True)
forecast_raw.head()

,index,date_time,temp,humid,wspd,wdeg,cloud,weather_id
0,0,2021-06-10 12:00:00,25.34,42,3.23,257,15,801
1,1,2021-06-10 15:00:00,24.94,46,2.67,252,15,801
2,2,2021-06-10 18:00:00,23.17,60,1.62,286,28,802
3,3,2021-06-10 21:00:00,16.79,90,2.47,346,100,804
4,6,2021-06-11 06:00:00,17.04,81,3.31,240,16,801


In [20]:
#To find the value with the most apperance
from collections import Counter

def most_frequent(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

In [21]:
def merge_data(weather):
    new_data = pd.DataFrame(columns = ['date',
                                       'mor_temp','mor_humid','mor_wspd','mor_wdeg','mor_cloud','mor_weather_id',
                                       'noo_temp','noo_humid','noo_wspd','noo_wdeg','noo_cloud','noo_weather_id',
                                       'aft_temp','aft_humid','aft_wspd','aft_wdeg','aft_cloud','aft_weather_id',])
    
    mor_count, mor_temp, mor_humid, mor_wspd, mor_wdeg, mor_cloud = [0]*6
    noo_count, noo_temp, noo_humid, noo_wspd, noo_wdeg, noo_cloud = [0]*6
    aft_count, aft_temp, aft_humid, aft_wspd, aft_wdeg, aft_cloud = [0]*6
    
    mor_weather_id=[]
    noo_weather_id=[]
    aft_weather_id=[]
    
    current_date = weather.loc[0, 'date_time'][0:10]
    
    for i in range(len(weather)):
        if weather.loc[i, 'date_time'][0:10] != current_date:
            new_data = new_data.append({
                'date':current_date,
                'mor_temp':mor_temp/mor_count,
                'mor_humid':mor_humid/mor_count,
                'mor_wspd':mor_wspd/mor_count,
                'mor_wdeg':mor_wdeg/mor_count,
                'mor_cloud':mor_cloud/mor_count,
                'mor_weather_id':most_frequent(mor_weather_id),
                'noo_temp':noo_temp/noo_count,
                'noo_humid':noo_humid/noo_count,
                'noo_wspd':noo_wspd/noo_count,
                'noo_wdeg':noo_wdeg/noo_count,
                'noo_cloud':noo_cloud/noo_count,
                'noo_weather_id':most_frequent(noo_weather_id),
                'aft_temp':aft_temp/aft_count,
                'aft_humid':aft_humid/aft_count,
                'aft_wspd':aft_wspd/aft_count,
                'aft_wdeg':aft_wdeg/aft_count,
                'aft_cloud':aft_cloud/aft_count,
                'aft_weather_id':most_frequent(aft_weather_id)
            }, ignore_index=True)
            
            mor_count, mor_temp, mor_humid, mor_wspd, mor_wdeg, mor_cloud = [0]*6
            noo_count, noo_temp, noo_humid, noo_wspd, noo_wdeg, noo_cloud = [0]*6
            aft_count, aft_temp, aft_humid, aft_wspd, aft_wdeg, aft_cloud = [0]*6
            
            mor_weather_id=[]
            noo_weather_id=[]
            aft_weather_id=[]
    
            current_date = weather.loc[i, 'date_time'][0:10]
        
        if int(weather.loc[i, 'date_time'][11:13]) < 11:
            mor_count += 1
            mor_temp += weather.loc[i, 'temp']
            mor_humid += weather.loc[i, 'humid']
            mor_wspd += weather.loc[i, 'wspd']
            mor_wdeg += weather.loc[i, 'wdeg']
            mor_cloud += weather.loc[i, 'cloud']
            mor_weather_id.append(weather['weather_id'][i])
            
        elif int(weather.loc[i, 'date_time'][11:13]) < 16:
            noo_count += 1
            noo_temp += weather.loc[i, 'temp']
            noo_humid += weather.loc[i, 'humid']
            noo_wspd += weather.loc[i, 'wspd']
            noo_wdeg += weather.loc[i, 'wdeg']
            noo_cloud += weather.loc[i, 'cloud']
            noo_weather_id.append(weather['weather_id'][i])
            
        else:
            aft_count += 1
            aft_temp += weather.loc[i, 'temp']
            aft_humid += weather.loc[i, 'humid']
            aft_wspd += weather.loc[i, 'wspd']
            aft_wdeg += weather.loc[i, 'wdeg']
            aft_cloud += weather.loc[i, 'cloud']
            aft_weather_id.append(weather['weather_id'][i])
        
    return new_data

In [22]:
forecast_data = merge_data(forecast_raw)

In [23]:
forecast_data.head()

,date,mor_temp,mor_humid,mor_wspd,mor_wdeg,mor_cloud,mor_weather_id,noo_temp,noo_humid,noo_wspd,noo_wdeg,noo_cloud,noo_weather_id,aft_temp,aft_humid,aft_wspd,aft_wdeg,aft_cloud,aft_weather_id
0,2021-06-10,25.340000,42.000000,3.230000,257.000000,15.000000,801,24.94,46.0,2.67,252.0,15.0,801,19.980,75.0,2.045,316.0,64.0,802
1,2021-06-11,20.936667,69.333333,4.360000,244.333333,20.333333,801,23.76,62.0,4.45,265.0,87.0,804,19.315,78.5,3.785,270.5,86.5,803
2,2021-06-12,19.143333,67.000000,4.626667,295.333333,97.000000,804,19.54,53.0,6.11,327.0,100.0,804,13.670,72.5,3.760,321.0,37.5,803
3,2021-06-13,17.486667,62.000000,1.940000,308.666667,4.666667,800,23.01,51.0,2.34,326.0,15.0,801,17.600,78.0,2.560,38.5,4.5,800


In [ ]:
forecast_data.info()

In [25]:
path = Path('.')

In [26]:
path.ls()

(#28) [Path('.~lock.OpenWeather_Antwerp.csv#'),Path('training_data_v3.csv'),Path('training_data_v3_1.csv'),Path('training_data_v2.csv'),Path('Timeseries_52.012_6.131_SA_0deg_-179deg_2005_2016_Doesburg.csv'),Path('EnergyProduction_LinearRegression_v2.2.ipynb'),Path('.git'),Path('SolarEnergyDataRefine_v3.ipynb'),Path('EnergyProduction_LinearRegression_v2.1.ipynb'),Path('SolarEnergy.zip')...]

In [27]:
learn = load_learner(path/'export_v2.pkl')

In [28]:
day_list = []
month_list = []
for i in range(len(forecast_data)):
    day_list.append(int(forecast_data.date[i][8:]))
    month_list.append(int(forecast_data.date[i][5:7]))
forecast_data['day'] = day_list
forecast_data['month'] = month_list

In [29]:
forecast_input = forecast_data.drop(['date'], axis=1, inplace=False)
forecast_input.head()

,mor_temp,mor_humid,mor_wspd,mor_wdeg,mor_cloud,mor_weather_id,noo_temp,noo_humid,noo_wspd,noo_wdeg,noo_cloud,noo_weather_id,aft_temp,aft_humid,aft_wspd,aft_wdeg,aft_cloud,aft_weather_id,day,month
0,25.340000,42.000000,3.230000,257.000000,15.000000,801,24.94,46.0,2.67,252.0,15.0,801,19.980,75.0,2.045,316.0,64.0,802,10,6
1,20.936667,69.333333,4.360000,244.333333,20.333333,801,23.76,62.0,4.45,265.0,87.0,804,19.315,78.5,3.785,270.5,86.5,803,11,6
2,19.143333,67.000000,4.626667,295.333333,97.000000,804,19.54,53.0,6.11,327.0,100.0,804,13.670,72.5,3.760,321.0,37.5,803,12,6
3,17.486667,62.000000,1.940000,308.666667,4.666667,800,23.01,51.0,2.34,326.0,15.0,801,17.600,78.0,2.560,38.5,4.5,800,13,6


In [30]:
predicted_value = []

for i in range(len(forecast_data)):
    row, clas, probs = learn.predict(forecast_input.iloc[i])
    predicted_value.append(clas)

In [31]:
predicted_value

[tensor([0.4472]), tensor([0.4932]), tensor([15.3369]), tensor([21.1662])]